In [5]:
import simpy
import random

# Constants
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_SERVICE_TIME = 5.0  # Min service time
MAX_SERVICE_TIME = 15.0  # Max service time

def customer(env, name, counter):
    """Customer arrives, is served and leaves."""
    arrival_time = env.now
    print(f'{name} arrives at the bank at {arrival_time:.2f}.')
    
    with counter.request() as req:
        yield req
        
        wait_time = env.now - arrival_time
        print(f'{name} waits for {wait_time:.2f} units.')
        
        service_time = random.uniform(MIN_SERVICE_TIME, MAX_SERVICE_TIME)
        print(f'{name} is being served for {service_time:.2f} units.')
        yield env.timeout(service_time)
        
        print(f'{name} leaves the bank at {env.now:.2f}.')

def customer_generator(env, counter):
    """Generate new customers at random intervals."""
    for i in range(NEW_CUSTOMERS):
        yield env.timeout(random.expovariate(1.0 / INTERVAL_CUSTOMERS))
        env.process(customer(env, f'Customer {i+1}', counter))

# Setup and start the simulation
print('Bank Simulation')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)
env.process(customer_generator(env, counter))

# Execute the simulation
env.run(until=40)


Bank Simulation
Customer 1 arrives at the bank at 10.20.
Customer 1 waits for 0.00 units.
Customer 1 is being served for 7.75 units.
Customer 2 arrives at the bank at 10.45.
Customer 3 arrives at the bank at 12.98.
Customer 1 leaves the bank at 17.95.
Customer 2 waits for 7.50 units.
Customer 2 is being served for 11.77 units.
Customer 4 arrives at the bank at 26.32.
Customer 2 leaves the bank at 29.72.
Customer 3 waits for 16.74 units.
Customer 3 is being served for 5.87 units.
Customer 3 leaves the bank at 35.59.
Customer 4 waits for 9.27 units.
Customer 4 is being served for 9.22 units.


In [6]:
import copy

In [7]:
env2 = clonify(env)

TypeError: cannot pickle 'generator' object

In [3]:
env.run()

Customer 4 leaves the bank at 44.81.
Customer 5 arrives at the bank at 48.59.
Customer 5 waits for 0.00 units.
Customer 5 is being served for 5.30 units.
Customer 5 leaves the bank at 53.89.


In [8]:
import simpy
import random
import copy

# Constants
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_SERVICE_TIME = 5.0  # Min service time
MAX_SERVICE_TIME = 15.0  # Max service time

class SimState:
    def __init__(self, env, counter, customers):
        self.env = env
        self.counter = counter
        self.customers = customers

    def save_state(self):
        """Save the state of the environment."""
        self.saved_env = copy.deepcopy(self.env)
        self.saved_counter = copy.deepcopy(self.counter)
        self.saved_customers = copy.deepcopy(self.customers)

    def restore_state(self):
        """Restore the state of the environment."""
        self.env = copy.deepcopy(self.saved_env)
        self.counter = copy.deepcopy(self.saved_counter)
        self.customers = copy.deepcopy(self.saved_customers)

def customer(env, name, counter):
    """Customer arrives, is served and leaves."""
    arrival_time = env.now
    print(f'{name} arrives at the bank at {arrival_time:.2f}.')
    
    with counter.request() as req:
        yield req
        
        wait_time = env.now - arrival_time
        print(f'{name} waits for {wait_time:.2f} units.')
        
        service_time = random.uniform(MIN_SERVICE_TIME, MAX_SERVICE_TIME)
        print(f'{name} is being served for {service_time:.2f} units.')
        yield env.timeout(service_time)
        
        print(f'{name} leaves the bank at {env.now:.2f}.')

def customer_generator(env, counter, customers):
    """Generate new customers at random intervals."""
    for i in range(NEW_CUSTOMERS):
        yield env.timeout(random.expovariate(1.0 / INTERVAL_CUSTOMERS))
        process = env.process(customer(env, f'Customer {i+1}', counter))
        customers.append(process)

def run_simulation():
    print('Bank Simulation')
    random.seed(RANDOM_SEED)

    # Create environment and start processes
    env = simpy.Environment()
    counter = simpy.Resource(env, capacity=1)
    customers = []
    env.process(customer_generator(env, counter, customers))

    # Return the environment and other components
    return env, counter, customers

# Run the simulation once
env, counter, customers = run_simulation()

# Create the simulation state handler
sim_state = SimState(env, counter, customers)

# Run the simulation up to a specified time and save the state
env.run(until=30)
sim_state.save_state()

# Restore the simulation state and continue running
sim_state.restore_state()
sim_state.env.run(until=60)


Bank Simulation
Customer 1 arrives at the bank at 10.20.
Customer 1 waits for 0.00 units.
Customer 1 is being served for 7.75 units.
Customer 2 arrives at the bank at 10.45.
Customer 3 arrives at the bank at 12.98.
Customer 1 leaves the bank at 17.95.
Customer 2 waits for 7.50 units.
Customer 2 is being served for 11.77 units.
Customer 4 arrives at the bank at 26.32.
Customer 2 leaves the bank at 29.72.
Customer 3 waits for 16.74 units.
Customer 3 is being served for 5.87 units.


TypeError: cannot pickle 'generator' object